In [1]:
import pandas as pd
from ortools.sat.python import cp_model
from utils import replicate_row,base_selection,expand_rows,clean_data,mock_teach_df
from teacher_alocation import TeacherScheduler

In [2]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/ROTA TESTE CARLOS E ALLAN.xlsx',header=1)

In [3]:

aulas_simples,doub,tri=base_selection(aulas_raw)

# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = expand_rows(doub, lambda row: replicate_row(row, times=2))
aulas_triplicadas = expand_rows(tri, lambda row: replicate_row(row, times=3))

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

df_final = clean_data(df_tratado)

df_final.head()

,Grupo,Horário,Dias da Semana,Unidade,STATUS,MOD,horario_tratado
0,MBA DAVI BRAGA,14:00:00,2ª,Vicentina,PRESENCIAL,MBA,1900-01-01 14:00:00
2,MBA LIGA DOS CAMPEÕES,18:30:00,5ª,Vicentina,PRESENCIAL,MBA,1900-01-01 18:30:00
4,MBA IDEIAS EM AÇÃO,19:00:00,2ª,Vicentina,PRESENCIAL,MBA,1900-01-01 19:00:00
6,GREENSBURG ONLINE,07:00:00,2ª,Vicentina,ONLINE,Grupo,1900-01-01 07:00:00
8,GREENSBURG ONLINE,07:00:00,3ª,Vicentina,ONLINE,Grupo,1900-01-01 07:00:00


In [4]:
info_professors = mock_teach_df(30)
info_professors.head()

,professor,tem_carro,fala_espanhol,tem_pc
0,Matthew,1,0,1
1,Tina,1,0,1
2,Kevin,1,1,1
3,Hector,1,0,0
4,William,1,1,1


In [5]:
Ts = TeacherScheduler(df_final, info_professors)
base_alocada = Ts.schedule_teachers()
base_alocada

,Professor,Grupo
0,Hector,MBA DAVI BRAGA
1,Matthew,MBA LIGA DOS CAMPEÕES
2,Tina,MBA IDEIAS EM AÇÃO
3,Matthew,GREENSBURG ONLINE
4,Hector,SAVANAH PRESENCIAL
...,...,...
142,Hector,NEBRASKA PRESENCIAL
143,Bryan,SCOTT ONLINE
144,Richard,MONTERREY ONLINE
145,Curtis,SAN JUAN ONLINE
